<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/khaiii_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/kakao/khaiii.git

fatal: destination path 'khaiii' already exists and is not an empty directory.


In [5]:
!cd khaiii
!mkdir build
!cd build

mkdir: cannot create directory ‘build’: File exists


In [0]:
import os 
os.chdir('/content/khaiii/')

In [7]:
!mkdir build

mkdir: cannot create directory ‘build’: File exists


In [0]:
os.chdir('/content/khaiii/build/')

In [9]:
!cmake ..

-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
-- [hunter] Calculating Config-SHA1
-- [hunter] HUNTER_ROOT: /root/.hunter
-- [hunter] [ Hunter-ID: 70287b1 | Toolchain-ID: f1a5aaa | Config-ID: dffbc08 ]
-- [hunter] BOOST_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 1.68.0-p1)
-- Boost version: 1.68.0
-- [hunter] CXXOPTS_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 2.1.1-pre)
-- [hunter] EIGEN_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 3.3.5)
-- [hunter] FMT_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 4.1.0)
-- [hunter] GTEST_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 1.8.0-hunter-p11)
-- [hunter] NLOHMANN_JSON_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 3.3.0)
-- [hunter] SPDLOG_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 0.16.3-p1)
-- Configuring done
-- Generating done
-- Build files have been written t

In [10]:
!make all

[ 65%] Built target obj_khaiii
[ 69%] Built target khaiii
[ 76%] Built target bin_khaiii
[100%] Built target test_khaiii


In [11]:
!make resource

Built target resource


In [12]:
os.chdir('/content/khaiii/build/')
!./bin/khaiii --rsc-dir=./share/khaiii

[2019-11-21 08:08:21.934] [Resource] [info] NN model loaded
[2019-11-21 08:08:21.934] [Preanal] [info] preanal dictionary opened
[2019-11-21 08:08:21.934] [ErrPatch] [info] errpatch dictionary opened
[2019-11-21 08:08:21.934] [Restore] [info] restore dictionary opened
[2019-11-21 08:08:21.934] [Resource] [info] PoS tagger opened
^C


In [13]:
!ctest

Test project /content/khaiii/build
    Start 1: test_khaiii
1/1 Test #1: test_khaiii ......................   Passed    0.02 sec

100% tests passed, 0 tests failed out of 1

Total Test time (real) =   0.02 sec


In [14]:
!make package_python

Run CPack packaging tool for source...
CPack: Create package using ZIP
CPack: Install projects
CPack: - Install directory: /content/khaiii
CPack: Create package
CPack: - package: /content/khaiii/build/khaiii-0.4.zip generated.
Built target package_python


In [15]:
os.chdir("/content/khaiii/build/package_python/")
!pip install .

Processing /content/khaiii/build/package_python
  Created wheel for khaiii: filename=khaiii-0.4-cp36-none-any.whl size=22892062 sha256=bf0a1940d00db4d453c3377b4a0117536c49a9f7ce9e4ea22c87c444be7502ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-8k4myfxh/wheels/b0/03/83/fb129110ddf28d31298b0731cd182d7d0f094935ae9bf8d4bc
Successfully built khaiii
  Found existing installation: khaiii 0.4
    Uninstalling khaiii-0.4:
      Successfully uninstalled khaiii-0.4


In [16]:
os.chdir("/content/")
!mkdir simmilar_work
os.chdir("/content/simmilar_work/")

mkdir: cannot create directory ‘simmilar_work’: File exists


In [0]:
os.chdir("/content/simmilar_work/")
import numpy as np
import pandas as pd

In [18]:
!ls

191101_테스트셋.xlsx


In [20]:
# 파일 임포팅 파트
from google.colab import files
uploaded = files.upload()

Saving 191101_테스트셋.xlsx to 191101_테스트셋 (1).xlsx


In [21]:
# 파일 DF로 읽기

RStext = pd.read_excel("191101_테스트셋.xlsx", encoding = "UTF-8")
del RStext["Unnamed: 0"]
RStext.columns

Index(['과제수행년도', '부처명', '부처명(다부처포함)', '사업명', '과제고유번호', '보안과제여부', '신규계속구분',
       '과제명-국문', '과학기술표준분류1-대', '과학기술표준분류가중치1', '총연구비_합계(원)', '요약문_연구목표',
       '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드', '요약문_영문키워드', '과제관리기관', '과제명-국문_길이',
       '요약문_연구목표_길이', '요약문_연구내용_길이', '요약문_기대효과_길이', '요약문_한글키워드_길이',
       '요약문_영문키워드_길이'],
      dtype='object')

In [22]:
# 통합텍스트 셀 생성
RStext["전체 텍스트"] = RStext['과제명-국문'] + " " + RStext['요약문_연구목표'] + " " + RStext['요약문_연구내용'] + " " + RStext['요약문_기대효과'] + " " + RStext['요약문_한글키워드'].str.replace(","," ") + " " + RStext['요약문_영문키워드'].str.replace(","," ")
RStext["전체 텍스트_기본"] = RStext["전체 텍스트"].str.findall('[a-z|A-Z|가-힣|&|.-.]+')
for i in range(len(RStext["전체 텍스트_기본"])):
  RStext["전체 텍스트_기본"][i] = " ".join(RStext["전체 텍스트_기본"][i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
print(RStext["전체 텍스트"][0])

In [0]:
import numpy as np
import pandas as pd
import re
from random import sample
from khaiii import KhaiiiApi
import time
api = KhaiiiApi()

In [0]:
Npercnt = 90

subj_for_test = pd.DataFrame()
subj_for_test["전체 텍스트"] = RStext["전체 텍스트"].str.split(" ")
subj_for_test["제목 길이"] = 0
subj_for_test["목표 길이"] = 0
subj_for_test["내용 길이"] = 0
subj_for_test["효과 길이"] = 0
subj_for_test["한키 길이"] = 0
subj_for_test["영키 길이"] = 0
subj_for_test["제목 추출"] = ''
subj_for_test["목표 추출"] = ''
subj_for_test["내용 추출"] = ''
subj_for_test["효과 추출"] = ''
subj_for_test["한키 추출"] = ''
subj_for_test["영키 추출"] = ''

In [0]:
print(subj_for_test["전체 텍스트"][0])

In [27]:
for i in range(1):
  # 텍스트와 길이 정의, 본문은 전체 텍스트를 띄어쓰기 형태로 붙였음. 
  sub_text = re.sub("\t|\n|\r","",str(" " + " ".join(subj_for_test["전체 텍스트"][i])+" "))
  subj_for_test["제목 길이"][i] = len(RStext["과제명-국문"][i]) + 1
  subj_for_test["목표 길이"][i] = int(subj_for_test["제목 길이"][i]) + len(RStext["요약문_연구목표"][i])
  subj_for_test["내용 길이"][i] = int(subj_for_test["목표 길이"][i]) + len(RStext["요약문_연구내용"][i])
  subj_for_test["효과 길이"][i] = int(subj_for_test["내용 길이"][i]) + len(RStext["요약문_기대효과"][i])
  subj_for_test["한키 길이"][i] = int(subj_for_test["효과 길이"][i]) + len(RStext["요약문_한글키워드"][i])
  subj_for_test["영키 길이"][i] = int(subj_for_test["한키 길이"][i]) + len(RStext["요약문_영문키워드"][i])
  # 형태소 분석, 아직 문서 하나
  test = []
  for word in api.analyze(RStext["전체 텍스트"][i]):
    test.append(str(word))
    test_str = re.sub('[A-Za-z가-힣]*\t|,'," "," ".join(test))
    test_str = re.compile('[A-Za-z가-힣]*/NNG|[A-Za-z가-힣]*/SL|[A-Za-z가-힣]*/NNP').findall(test_str)
    test_str = re.sub('/NNG|/SL|/NNP',""," ".join(test_str))
    test_str = re.compile('[A-Za-z가-힣][A-Za-z가-힣]+').findall(test_str)
    test_set = list(set(test_str))
    test_str = str(" "+" ".join(test_str)+" ")
    subj_count = pd.DataFrame(np.zeros([len(test_set),2]),columns = ["키워드","빈도"])
    subj_count["위치"] = ""
    subj_count["위치변화"] = ""
    subj_count["변경수"] = 0
    subj_count["변경대상"] = ""
    subj_count["대체단어"] = "연구"
    subj_count["더미글자"] = "뷁"
    for k in range(len(test_set)):
      subj_count["키워드"][k] = test_set[k]
      subj_count["빈도"][k] = len(re.compile(str(" "+test_set[k]+" ")).findall(test_str))
      subj_count["위치"][k] = [m.start() for m in re.finditer(subj_count["키워드"][k],sub_text)]
      subj_count["위치변화"][k] = sample(subj_count["위치"][k],len(subj_count["위치"][k]))
    subj_count = subj_count.sort_values(by = ["빈도"],ascending=False).reset_index(drop=True)
    subj_count["대체단어길이"] = subj_count["키워드"].str.len()-2
    Keycount = int(round(sum(subj_count["빈도"])/100*(100-Npercnt),0))
    for j in range(len(subj_count)):
      subj_count["변경수"][j] = int(np.random.randint(0, high = int(subj_count["빈도"][j])+1,size = 1))*int(sample([0,1,1,1],1)[0])
      if Keycount < 0:
        subj_count["변경수"][j] = subj_count["변경수"][j] - (0 - Keycount)
        subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
        break
      subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
  for l in range(len(subj_count)):
    if len(subj_count["변경대상"][l]) == 0:
      continue
    else:
      for ll in range(len(subj_count["변경대상"][l])):
        sub_text = "".join((sub_text[:subj_count["변경대상"][l][ll]],"연구",sub_text[subj_count["변경대상"][l][ll]+len(subj_count["키워드"][l]):]))
        if subj_count["변경대상"][l][ll] <= subj_for_test["제목 길이"][i]:
          subj_for_test["제목 길이"][i] = subj_for_test["제목 길이"][i] - (len(subj_count["키워드"][l])-2)
        elif subj_count["변경대상"][l][ll] <= subj_for_test["목표 길이"][i]:
          subj_for_test["목표 길이"][i] = subj_for_test["목표 길이"][i] - (len(subj_count["키워드"][l])-2)
        elif subj_count["변경대상"][l][ll] <= subj_for_test["내용 길이"][i]:
          subj_for_test["내용 길이"][i] = subj_for_test["내용 길이"][i] - (len(subj_count["키워드"][l])-2)
        elif subj_count["변경대상"][l][ll] <= subj_for_test["효과 길이"][i]:
          subj_for_test["효과 길이"][i] = subj_for_test["효과 길이"][i] - (len(subj_count["키워드"][l])-2)
        elif subj_count["변경대상"][l][ll] <= subj_for_test["한키 길이"][i]:
          subj_for_test["한키 길이"][i] = subj_for_test["한키 길이"][i] - (len(subj_count["키워드"][l])-2)
        elif subj_count["변경대상"][l][ll] <= subj_for_test["영키 길이"][i]:
          subj_for_test["영키 길이"][i] = subj_for_test["영키 길이"][i] - (len(subj_count["키워드"][l])-2)
        print(sub_text)
  subj_for_test["제목 추출"][i] = sub_text[0:subj_for_test["제목 길이"][i]]
  subj_for_test["목표 추출"][i] = sub_text[subj_for_test["제목 길이"][i]:subj_for_test["목표 길이"][i]]
  subj_for_test["내용 추출"][i] = sub_text[subj_for_test["목표 길이"][i]:subj_for_test["내용 길이"][i]]
  subj_for_test["효과 추출"][i] = sub_text[subj_for_test["내용 길이"][i]:subj_for_test["효과 길이"][i]]
  subj_for_test["한키 추출"][i] = sub_text[subj_for_test["효과 길이"][i]:subj_for_test["한키 길이"][i]]
  subj_for_test["영키 추출"][i] = sub_text[subj_for_test["한키 길이"][i]:subj_for_test["영키 길이"][i]]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: Setti

 바이러스 현장 검출용 초소형화 펜타입 표면 플라즈몬 공명 시스템 개발 ○ 본 제안기술은 SPR 시스템 기술, 광학 집적 기술, 소프트웨어 기술, 바이오컨쥬게이셔 기술을 바탕으로 하여, 바이러스 현장 진단용 초소형화 펜타입 SPR 시스템을 세계최초로 개발하는 것을 주요 골자로 함.○ 본 제안기술은 Kretchmann configuration 기반의 SPR 센싱 방법을 주요 적용 기술로서 활용하지만, 추가적으로 최근 연구가 활발히 진행되고 있는 나노 플라즈모닉 센싱 기반으로 개발 범위를 확장하여 학술적으로도 깊이 있는 신규의 센싱 기법을 개발하고자 함. ○ 초소형화 펜타입 SPR 시스템 설계/제작 기술 연구- Kretchmann configuration 기반 광학 미세 집적 기술 연구- 나노플라즈모닉 기반 광학 미세 집적 연구 연구- 일회용 칩 탈부착 기술 및 액체유입 차단 기술 연구- 센서 분해능: 1 x 10-7 RIU (refractive index unit)- SPR 광학모듈 크기: 15 x 15 x 50 mm (W x D x H)- 무선, 저전력 시스템 집적 기술 개발○ 일회용 칩 설계/제작 기술 및 바이러스 측정 기술 연구- 프리즘 일체형 칩 설계 및 최적화 (Kretchmann configuration)- 나노물질 (금속입자 및 금속선) 패터닝 기술 연구 (나노플라즈모닉)- 면역반응 기반 바이러스 측정 방법 최적화- Aptamer, peptide 등 초고민감도, 고선택성을 갖는 경제성 최적 리셉터 발굴- 시료농축기술이 적용된 시료전처리 카트리지 개발○ 스마트폰 연동 소프트웨어 기술 연구- Matlab 기반 SPR curve fitting 알고리즘 단순화 기술 연구- 스마트폰 통신 기술 연구- 바이러스 정량 진단 어플리케이션 시작품 개발○ 임상 유효성 검증 및 실용화 기술 연구 - 대표 급성감염 바이러스 1종에 대한 임상 유효성 검증- 민감도 (sensitivity): 분자진단의 90%- 선택성 (selectivity): 90% 이상 (최종 상용화 단

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be s

In [0]:
test = []
start = time.time()
for word in api.analyze(RStext["전체 텍스트"][i]):
  test.append(str(word))
  np.random.randint(0,1000000,1000)
print("time :", time.time() - start)

In [0]:
for i in range(1):
  sub_text = re.sub("\t|\n|\r","",str(" " + " ".join(subj_for_test["전체 텍스트"][i])+" "))
  subj_for_test["제목 길이"][i] = len(RStext["과제명-국문"][i]) + 1
  subj_for_test["목표 길이"][i] = int(subj_for_test["제목 길이"][i]) + len(RStext["요약문_연구목표"][i])
  subj_for_test["내용 길이"][i] = int(subj_for_test["목표 길이"][i]) + len(RStext["요약문_연구내용"][i])
  subj_for_test["효과 길이"][i] = int(subj_for_test["내용 길이"][i]) + len(RStext["요약문_기대효과"][i])
  subj_for_test["한키 길이"][i] = int(subj_for_test["효과 길이"][i]) + len(RStext["요약문_한글키워드"][i])
  subj_for_test["영키 길이"][i] = int(subj_for_test["한키 길이"][i]) + len(RStext["요약문_영문키워드"][i])
  test = []
  for word in api.analyze(RStext["전체 텍스트"][i]):
    test.append(str(word))
    test_str = re.sub('[A-Za-z가-힣]*\t|,'," "," ".join(test))
    test_str = re.compile('[A-Za-z가-힣]*/NNG|[A-Za-z가-힣]*/SL|[A-Za-z가-힣]*/NNP').findall(test_str)
    test_str = re.sub('/NNG|/SL|/NNP',""," ".join(test_str))
    test_str = re.compile('[A-Za-z가-힣][A-Za-z가-힣]+').findall(test_str)
    test_set = list(set(test_str))
    test_str = str(" "+" ".join(test_str)+" ")
    subj_count = pd.DataFrame(np.zeros([len(test_set),2]),columns = ["키워드","빈도"])
    subj_count["위치"] = ""
    subj_count["위치변화"] = ""
    subj_count["변경수"] = 0
    subj_count["변경대상"] = ""
    subj_count["대체단어"] = "연구"
    subj_count["더미글자"] = "뷁"
    for k in range(len(test_set)):
      subj_count["키워드"][k] = test_set[k]
      subj_count["빈도"][k] = len(re.compile(str(" "+test_set[k]+" ")).findall(test_str))
      subj_count["위치"][k] = [m.start() for m in re.finditer(subj_count["키워드"][k],sub_text)]
      subj_count["위치변화"][k] = sample(subj_count["위치"][k],len(subj_count["위치"][k]))


In [0]:
print(subj_for_test["전체 텍스트"][i])
print(sub_text)
print(subj_for_test["제목 추출"][i])
print(subj_for_test["목표 추출"][i])
print(subj_for_test["내용 추출"][i])
print(subj_for_test["효과 추출"][i])
print(subj_for_test["한키 추출"][i])
print(subj_for_test["영키 추출"][i])

In [0]:
subj_for_test.head(1)

In [0]:
sub_text
[m.start() for m in re.finditer(subj_count["키워드"][k],sub_text)]

In [0]:
subj_count.head()

In [0]:
# # 
# test_str = re.sub('[A-Za-z가-힣]*\t'," "," ".join(test))
# test_str = re.compile('[A-Za-z가-힣]*/NNG|[A-Za-z가-힣]*/SL|[A-Za-z가-힣]*/NNP').findall(test_str)
# test_str = re.sub('/NNG|/SL|/NNP',""," ".join(test_str))
# test_str = re.compile('[A-Za-z가-힣][A-Za-z가-힣]+').findall(test_str)
# test_set = list(set(test_str))
# test_str = str(" "+" ".join(test_str)+" ")
# print(test_str)
# print(test_set)

In [0]:
# subj_count = pd.DataFrame(np.zeros([len(test_set),2]),columns = ["키워드","빈도"])
# subj_count["위치"] = ""
# subj_count["위치변화"] = ""
# subj_count["변경수"] = 0
# subj_count["변경대상"] = ""
# for i in range(len(test_set)):
#   subj_count["키워드"][i] = test_set[i]
#   subj_count["빈도"][i] = len(re.compile(str(" "+test_set[i]+" ")).findall(test_str))
#   subj_count["위치"][i] = list(np.where(np.array(test_str.split(" ")) == test_set[i])[0])
#   subj_count["위치변화"][i] = sample(subj_count["위치"][i],len(subj_count["위치"][i]))
# subj_count = subj_count.sort_values(by = ["빈도"],ascending=False).reset_index(drop=True)
# Npercnt = 10
# Keycount = int(round(sum(subj_count["빈도"])/100*Npercnt,0))
# for j in range(len(subj_count)):
#   subj_count["변경수"][j] = int(np.random.randint(0, high = int(subj_count["빈도"][j])+1,size = 1))*int(sample([0,1,1,1],1)[0])
#   Keycount -= subj_count["변경수"][j]
#   if Keycount < 0:
#     subj_count["변경수"][j] = subj_count["변경수"][j] - (0 - Keycount)
#     subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
#     break
#   subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])


In [0]:
subj_count.sort_values(by =["빈도"],ascending=False)[0:30]

In [0]:
print(sub_text[382])
print(subj_count["키워드"][l])
print(subj_count["변경대상"][l][ll])
print(l)
print(ll)

In [0]:
print(int(round(sum(subj_count["빈도"])/100*Npercnt,0)))
print(sum(subj_count["변경수"]))

In [0]:
print(int(round(sum(subj_count["빈도"])/10*2,0)))

In [0]:
print(RStext["과제고유번호"][num])
print(RStext["과제명-국문"][num])

In [0]:
# 단어 체인지 후에 각 필드로 배분하기 위한 방법
print(len(RStext["전체 텍스트"][num].split(" ")))
print(len(RStext["과제명-국문"][num].split(" ")))
print(RStext["전체 텍스트"][num].split(" ")[0:len(RStext["과제명-국문"][num].split(" "))])
print(RStext["과제명-국문"][num].split(" "))
print(RStext["전체 텍스트"][num].split(" ")[len(RStext["과제명-국문"][num].split(" ")):(len(RStext["과제명-국문"][num].split(" "))+len(RStext["요약문_연구목표"][num].split(" ")))])
print(RStext["요약문_연구목표"][num].split(" "))
print(RStext["전체 텍스트"][num].split(" ")[len(RStext["과제명-국문"][num].split(" ")):(len(RStext["과제명-국문"][num].split(" "))+len(RStext["요약문_연구목표"][num].split(" ")))])

In [0]:
print(RStext["요약문_연구내용"][num].split(" "))

In [0]:
np.where(np.array(RStext["요약문_연구내용"][num].split(" ")) == "분석")[0]